In [1]:
import numpy as np
import xgboost
import pandas as pd
import re
import csv

from collections import defaultdict
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier

In [2]:
df = pd.read_csv("./validation.csv")
#df_valid = pd.read_csv("~/webecon/dataset/validation.csv")
#df_test = pd.read_csv("~/webecon/dataset/test.csv")

In [3]:
df_valid = pd.read_csv("./validation.csv")
df_test = pd.read_csv("./test.csv")

In [4]:
train_split = np.array_split(df, 1)
train = train_split[0]
train_y = train['click']

In [5]:
cols = ['click', 'weekday', 'hour', 'bidid', #'logtype', 
        'userid', 'useragent',
       'IP', 'region', 'city', 'adexchange', 'domain', 'url', 'urlid',
       'slotid', 'slotwidth', 'slotheight', 'slotvisibility', 'slotformat',
       'slotprice', 'creative', 'payprice', 'keypage',
       'advertiser', 'usertag', 'bidprice']

train_x = train[cols]

In [6]:
label_encoder = LabelEncoder()
vectorizer = DictVectorizer()

In [7]:
def get_browser(useragent):
    useragent_data = re.split('_',useragent)
    return useragent_data[1]

In [8]:
def get_os(useragent):
    useragent_data = re.split('_',useragent)
    return useragent_data[0]

In [9]:
def bid_request_features(bid,have_click):
    result = defaultdict(float)
    result['weekday=' + str(bid['weekday'].item())] += 1.0
    result['hour=' + str(bid['hour'].item())] += 1.0
    result['userid=' + bid['userid']] += 1.0
    result['broswer=' + get_browser(bid['useragent'])] += 1.0
    result['os=' + get_os(bid['useragent'])] += 1.0
    result['IP=' + bid['IP']] += 1.0
    result['region=' + str(bid['region'].item())] += 1.0
    result['city=' + str(bid['city'].item())] += 1.0
    result['adexchange=' + str(bid['adexchange'])] += 1.0
    result['domain=' + str(bid['domain'])] += 1.0
    result['url=' + str(bid['url'])] += 1.0
    result['slotid=' + bid['slotid']] += 1.0
    result['slotwidth=' + str(bid['slotwidth'].item())] += 1.0
    result['slotheight=' + str(bid['slotheight'].item())] += 1.0
    result['slotvisibility=' + bid['slotvisibility']] += 1.0
    result['slotformat=' + bid['slotformat']] += 1.0
    result['slotprice=' + str(bid['slotprice'].item())] += 1.0
    result['creative=' + bid['creative']] += 1.0
    result['keypage=' + str(bid['keypage'])] += 1.0 
    result['advertiser=' + str(bid['advertiser'].item())] += 1.0

   # usertag_data = re.split(',',bid['usertag'])
#     for tag in usertag_data:
#         result['usertag=' + str(tag)] += 1.0
        
    if have_click == 1:
        result['click=' + str(bid['click'].item())] += 1.0
        
    return result

In [10]:
train_bid_x = vectorizer.fit_transform([bid_request_features(train_x.iloc[i],0) for i in range(len(train_x))])
train_bid_y = label_encoder.fit_transform([train_x.iloc[i]['click'] for i in range(len(train_x))]) 

In [11]:
model = GradientBoostingClassifier()
model = model.fit(train_bid_x, train_bid_y)

In [12]:
bid_y= []

def predict_click_probab(test_bids):
    bid_x = vectorizer.transform([bid_request_features(test_bids.iloc[i],0) for i in range(len(test_bids))])
    bid_y.append(model.predict_proba(bid_x.toarray()))
    return bid_y

In [13]:
valid_cols = ['weekday', 'bidid', 'hour', 'userid', 'useragent',
       'IP', 'region', 'city', 'adexchange', 'domain', 'url', 'urlid',
       'slotid', 'slotwidth', 'slotheight', 'slotvisibility', 'slotformat',
       'slotprice', 'creative', 'keypage',
       'advertiser', 'usertag']

In [14]:
valid_split = np.array_split(df_valid, 1)
valid = valid_split[0]
valid_x = valid[valid_cols]

In [ ]:
click_valid_guess = predict_click_probab(valid_x)

In [ ]:
valid_x['bidid'] = valid['bidid']
valid_x['pred_click'] = click_valid_guess[0].tolist()

In [ ]:
with open("out.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for i in range(0,len(valid_x)):
        writer.writerow([valid_x['bidid'][i], valid_x['pred_click'][i][0], valid_x['pred_click'][i][1]])